# Tempo de execução do processo de ETL

In [ ]:
import time

tempoInicio = time.time()

## Código

tempoExecucao = time.time() - tempoInicio
print("Tempo de execução em segundos: %f" % tempoExecucao)

## Extração
Parte somente da extração dos dados, sem a transformação e carga.

In [ ]:
import time

tempoInicio = time.time()

In [ ]:
from pysus.ftp.databases.sih import SIH

In [ ]:
sih = SIH().load()

### Limitando os arquivos apenas para estimarmos valores
Grupo de AIH reduzidas e apenas para o Estado de São Paulo.

In [ ]:
group = "RD"
uf = "SP"

month = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
year = [2024]

In [ ]:
files = sih.get_files(group, uf, year, month)

print(f"O numero de arquivos eh: {len(files)}.")

files

### Fazendo o download dos dados em formato parquet

Fazendo download de apenas onze arquivos.

In [ ]:
path = '/home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIH/stage/parquet/2024/'

In [ ]:
for file in files:
    sih.download(file, local_dir = path)

In [ ]:
tempoFinalExtracao = time.time() - tempoInicio

In [ ]:
print(f"Totalizando, assim, {int(tempoFinalExtracao/60)} minutos e {(tempoFinalExtracao % 60):.2f} segundos.")

## Transformação
Para fazer as transformações, primeiro vamos carregar os dados e depois vamos fazer a transformação.

In [ ]:
import time

tempoInicio = time.time()

In [ ]:
import pandas as pd
import os

In [ ]:
group = "RD"
uf = "SP"

year = 2015
months = [1, 2, 3]

path = f'/home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIH/stage/parquet/{year}/'

In [ ]:
files = [f"{group}{uf}{year-2000}{str(month).zfill(2)}.parquet" for month in months]

files

Devido à problemas com o WSL, tive que fazer o carregamento de parte dos dados, não integralmente os 12 arquivos do ano.

In [ ]:
dfs = []

# Fazendo o carregamento de apenas dos meses 1, 2 e 3
for file in files:
    full_path = os.path.join(path, file)
    df = pd.read_parquet(full_path)
    dfs.append(df)

### Mostrando os dados carregados

In [ ]:
dfs[0].head()

In [ ]:
dfs[1].head()

In [ ]:
dfs[2].head()

### Separando os dados em suas respectivas tabelas

In [ ]:
dimensoes = {   'Dimensão Tempo': 
                    ['ANO_CMPT', 'MES_CMPT', 'DT_INTER', 'DT_SAIDA'],
                    
                'Dimensão Localização': 
                    ['CEP', 'MUNIC_RES', 'MUNIC_MOV'],
                    
                'Dimensão Hospital': 
                    ['CGC_HOSP', 'CNES'],
                    
                'Dimensão Paciente': 
                    ['NASC', 'SEXO', 'IDADE', 'COD_IDADE', 'NACIONAL', 'INSTRU', 'RACA_COR', 'ETNIA', 'CBOR', 'MORTE'],
                    
                'Dimensão UTI': 
                    ['UTI_MES_TO', 'MARCA_UTI', 'VAL_UTI'],
                    
                'Dimensão Procedimento': 
                    ['PROC_SOLIC', 'PROC_REA', 'VAL_SH', 'VAL_SP', 'N_AIH', 'VAL_TOT', 'INFEHOSP'],
                    
                'Dimensão Diagnóstico': 
                    ['DIAG_PRINC', 'DIAG_SECUN', 'DIAGSEC1', 'DIAGSEC2', 'DIAGSEC3', 'DIAGSEC4', 'DIAGSEC5', 
                     'DIAGSEC6', 'DIAGSEC7', 'DIAGSEC8', 'DIAGSEC9', 'CID_MORTE', 'CID_ASSO']
                }

'''
    Atributos retirados: IND_VDRL, possivelmente INFEHOSP (verificar se estão vazios).
    Atributos adicionados: CID_ASSO.
'''

In [ ]:
# Funcao que separa as dimensoes de um DataFrame
def separaDimensoes(dataFrame, dimensoes):
    dfs = {}
    
    for dim in dimensoes:
        # Cria um DataFrame com as colunas da dimensão
        dfs[dim] = dataFrame[dimensoes[dim]]
    
    return dfs

In [ ]:
# Criando uma lista de listas de DataFrames. Cada lista deve armazenar os DataFrames de um mês e suas sublistas as dimensões.
dfs_separados_meses = []

for df in dfs:
    dfs_separados_meses.append(separaDimensoes(df, dimensoes))

### Mostrando os dados após a divisão em dimensões e fatos

In [ ]:
dfs_separados_meses[0]['Dimensão Tempo'].head()

In [ ]:
dfs_separados_meses[0]['Dimensão Localização'].head()

In [ ]:
dfs_separados_meses[1]['Dimensão Tempo'].head()

In [ ]:
dfs_separados_meses[2]['Dimensão Tempo'].head()

### Mostrando o tempo de execução do processo de transformação

In [ ]:
tempoFinalTransformacao = time.time() - tempoInicio

print(f"Tempo de transformação: {int(tempoFinalTransformacao/60)} minutos e {(tempoFinalTransformacao % 60):.2f} segundos.")

## Carga
Aqui vamos inserir os dados transformados em um banco de dados.

### Criando uma classe que contém as informacoes de todas as dimensoes

In [ ]:
import pandas as pd
import sqlalchemy as sa

In [ ]:
df = pd.read_csv('/home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIH/csv/concat/fev-200_amostras_ano.csv', sep = ',', encoding='latin-1')

In [ ]:
df.head()

In [ ]:
# Criando uma classe que instancia tabelas de dimensões do SIH

class tabelas:
    def __init__(self, dimensao):
        
        lista_dimensoes = ['Dimensão Tempo', 'Dimensão Localização', 'Dimensão Hospital', 'Dimensão Paciente', 
                           'Dimensão UTI', 'Dimensão Procedimento', 'Dimensão Diagnóstico', 'Tabela Fato Internação']
        
        if dimensao not in lista_dimensoes:
            raise ValueError('Dimensão inválida.')
        else:
            self.dimensao = dimensao
            
        self.colunas = []
        if self.dimensao == 'Dimensão Tempo':
            self.colunas = ['ANO_CMPT', 'MES_CMPT', 'DT_INTER', 'DT_SAIDA']
        elif self.dimensao == 'Dimensão Localização':
            self.colunas = ['CEP', 'MUNIC_RES', 'MUNIC_MOV']
        elif self.dimensao == 'Dimensão Hospital':
            self.colunas = ['CGC_HOSP', 'CNES']
        elif self.dimensao == 'Dimensão Paciente':
            self.colunas = ['NASC', 'SEXO', 'IDADE', 'COD_IDADE', 'NACIONAL', 'INSTRU', 'RACA_COR', 'ETNIA', 'CBOR', 'MORTE']
        elif self.dimensao == 'Dimensão UTI':
            self.colunas = ['UTI_MES_TO', 'MARCA_UTI', 'VAL_UTI']
        elif self.dimensao == 'Dimensão Procedimento':
            self.colunas = ['PROC_SOLIC', 'PROC_REA', 'VAL_SH', 'VAL_SP', 'N_AIH', 'VAL_TOT', 'INFEHOSP']
        elif self.dimensao == 'Dimensão Diagnóstico':
            self.colunas = ['DIAG_PRINC', 'DIAG_SECUN', 'DIAGSEC1', 'DIAGSEC2', 'DIAGSEC3', 'DIAGSEC4', 'DIAGSEC5', 
                            'DIAGSEC6', 'DIAGSEC7', 'DIAGSEC8', 'DIAGSEC9', 'CID_MORTE', 'CID_ASSO']
        elif self.colunas == 'Tabela Fato Internação':
            self.colunas = ['N_AIH', 'NASC', 'SEXO', 'IDADE', 'COD_IDADE', 'NACIONAL', 'INSTRU', 'RACA_COR', 'ETNIA', 'MUNIC_MOV', 
                            'CBOR', 'MARCA_UTI', 'DIAG_PRINC', 'DIAG_SECUN', 'DIAGSEC1', 'DIAGSEC2', 'DIAGSEC3', 'DIAGSEC4', 'DIAGSEC5', 
                            'DIAGSEC6', 'DIAGSEC7', 'DIAGSEC8', 'DIAGSEC9', 'VAL_SH', 'VAL_SP', 'VAL_UTI', 'VAL_TOT', 'INFEHOSP', 'MORTE']
            
        
    def __str__(self):
        return f'Tabela de {self.dimensao}'
    
    def get_dimensao(self):
        return self.dimensao
    
    def get_colunas(self):
        return self.colunas
    
    def insere_dados(self, df, conn, namedb):
        if self.dimensao == 'Dimensão Tempo':
            self.insere_dados_tempo(df, conn, namedb)
        elif self.dimensao == 'Dimensão Localização':
            self.insere_dados_localizacao(df, conn, namedb)
        elif self.dimensao == 'Dimensão Hospital':
            self.insere_dados_hospital(df, conn, namedb)
        elif self.dimensao == 'Dimensão Paciente':
            self.insere_dados_paciente(df, conn, namedb)
        elif self.dimensao == 'Dimensão UTI':
            self.insere_dados_uti(df, conn, namedb)
        elif self.dimensao == 'Dimensão Procedimento':
            self.insere_dados_procedimento(df, conn, namedb)
        elif self.dimensao == 'Dimensão Diagnóstico':
            self.insere_dados_diagnostico(df, conn, namedb)
        else:
            raise ValueError('Essa tabela não é de dimensão conhecida.')
        
    def insere_dados_tempo(self, df, conn, namedb):
        if self.dimensao == 'Dimensão Tempo':
            df.to_sql(namedb, conn, if_exists='append', index=False, dtype={
                'ANO_CMPT': sa.VARCHAR(4),
                'MES_CMPT': sa.VARCHAR(2),
                'DT_INTER': sa.VARCHAR(8),
                'DT_SAIDA': sa.VARCHAR(8)}
            )
        else:
            raise ValueError('Essa tabela não é de dimensão tempo.')
        
    def insere_dados_localizacao(self, df, conn, namedb):
        if self.dimensao == 'Dimensão Localização':
            df.to_sql(namedb, conn, if_exists='append', index=False, dtype={
                'CEP': sa.VARCHAR(8),
                'MUNIC_RES': sa.VARCHAR(6),
                'MUNIC_MOV': sa.VARCHAR(6)}
            )
        else:
            raise ValueError('Essa tabela não é de dimensão localização.')
        
    def insere_dados_hospital(self, df, conn, namedb):
        if self.dimensao == 'Dimensão Hospital':
            df.to_sql(namedb, conn, if_exists='append', index=False, dtype={
                'CGC_HOSP': sa.VARCHAR(16),
                'CNES': sa.VARCHAR(7)}
            )
        else:
            raise ValueError('Essa tabela não é de dimensão hospital.')
        
    def insere_dados_paciente(self, df, conn, namedb):
        if self.dimensao == 'Dimensão Paciente':
            df.to_sql(namedb, conn, if_exists='append', index=False, dtype={
                'NASC': sa.VARCHAR(8),
                'SEXO': sa.VARCHAR(1),
                'IDADE': sa.NUMERIC(2),
                'COD_IDADE': sa.VARCHAR(1),
                'NACIONAL': sa.VARCHAR(3),
                'INSTRU': sa.VARCHAR(1),
                'RACA_COR': sa.VARCHAR(2),
                'ETNIA': sa.VARCHAR(4),
                'CBOR': sa.VARCHAR(6),
                'MORTE': sa.NUMERIC(1)}
            )
        else:
            raise ValueError('Essa tabela não é de dimensão paciente.')
        
    def insere_dados_uti(self, df, conn, namedb):
        if self.dimensao == 'Dimensão UTI':
            df.to_sql(namedb, conn, if_exists='append', index=False, dtype={
                'UTI_MES_TO': sa.NUMERIC(3),
                'MARCA_UTI': sa.VARCHAR(2),
                'VAL_UTI': sa.NUMERIC(9,2)}
            )
        else:
            raise ValueError('Essa tabela não é de dimensão UTI.')
        
    def insere_dados_procedimento(self, df, conn, namedb):
        if self.dimensao == 'Dimensão Procedimento':
            df.to_sql(namedb, conn, if_exists='append', index=False, dtype={
                'PROC_SOLIC': sa.VARCHAR(10),
                'PROC_REA': sa.VARCHAR(10),
                'VAL_SH': sa.NUMERIC(13,2),
                'VAL_SP': sa.NUMERIC(13,2),
                'N_AIH': sa.VARCHAR(13),
                'VAL_TOT': sa.NUMERIC(14,2),
                'INFEHOSP': sa.VARCHAR(1)}
            )
        else:
            raise ValueError('Essa tabela não é de dimensão procedimento.')
        
    def insere_dados_diagnostico(self, df, conn, namedb):
        if self.dimensao == 'Dimensão Diagnóstico':
            df.to_sql(namedb, conn, if_exists='append', index=False, dtype={
                'DIAG_PRINC': sa.VARCHAR(4),
                'DIAG_SECUN': sa.VARCHAR(4),
                'DIAGSEC1': sa.VARCHAR(4),
                'DIAGSEC2': sa.VARCHAR(4),
                'DIAGSEC3': sa.VARCHAR(4),
                'DIAGSEC4': sa.VARCHAR(4),
                'DIAGSEC5': sa.VARCHAR(4),
                'DIAGSEC6': sa.VARCHAR(4),
                'DIAGSEC7': sa.VARCHAR(4),
                'DIAGSEC8': sa.VARCHAR(4),
                'DIAGSEC9': sa.VARCHAR(4),
                'CID_MORTE': sa.VARCHAR(4),
                'CID_ASSO': sa.VARCHAR(4)}
            )
        else:
            raise ValueError('Essa tabela não é de dimensão diagnóstico.')
        
        
    def cria_tabela(self, conn, namedb):
        metadata = sa.MetaData()
        metadata.reflect(bind=conn)
        
        if namedb in metadata.tables:
            print(f'Tabela {namedb} já existe.')
            return
        
        if self.dimensao == 'Dimensão Tempo':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('ANO_CMPT', sa.VARCHAR(4)),
                    sa.Column('MES_CMPT', sa.VARCHAR(2)),
                    sa.Column('DT_INTER', sa.VARCHAR(8)),
                    sa.Column('DT_SAIDA', sa.VARCHAR(8))
            ).create(conn)
        elif self.dimensao == 'Dimensão Localização':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('CEP', sa.VARCHAR(8)),
                    sa.Column('MUNIC_RES', sa.VARCHAR(6)),
                    sa.Column('MUNIC_MOV', sa.VARCHAR(6))
            ).create(conn)
        elif self.dimensao == 'Dimensão Hospital':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('CGC_HOSP', sa.VARCHAR(16)),
                    sa.Column('CNES', sa.VARCHAR(7))
            ).create(conn)
        elif self.dimensao == 'Dimensão Paciente':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('NASC', sa.VARCHAR(8)),
                    sa.Column('SEXO', sa.VARCHAR(1)),
                    sa.Column('IDADE', sa.NUMERIC(2)),
                    sa.Column('COD_IDADE', sa.VARCHAR(1)),
                    sa.Column('NACIONAL', sa.VARCHAR(3)),
                    sa.Column('INSTRU', sa.VARCHAR(1)),
                    sa.Column('RACA_COR', sa.VARCHAR(2)),
                    sa.Column('ETNIA', sa.VARCHAR(4)),
                    sa.Column('CBOR', sa.VARCHAR(6)),
                    # sa.Column('CBOR', sa.VARCHAR(3)), // O dicionário está com 3 caracteres, mas os dados apresentam 6.
                    sa.Column('MORTE', sa.NUMERIC(1))
            ).create(conn)
        elif self.dimensao == 'Dimensão UTI':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('UTI_MES_TO', sa.NUMERIC(3)),
                    sa.Column('MARCA_UTI', sa.VARCHAR(2)),
                    sa.Column('VAL_UTI', sa.NUMERIC(9,2))
            ).create(conn)
        elif self.dimensao == 'Dimensão Procedimento':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('PROC_SOLIC', sa.VARCHAR(10)),
                    sa.Column('PROC_REA', sa.VARCHAR(10)),
                    sa.Column('VAL_SH', sa.NUMERIC(13,2)),
                    sa.Column('VAL_SP', sa.NUMERIC(13,2)),
                    sa.Column('N_AIH', sa.VARCHAR(13)),
                    sa.Column('VAL_TOT', sa.NUMERIC(14,2)),
                    sa.Column('INFEHOSP', sa.VARCHAR(1))
            ).create(conn)
        elif self.dimensao == 'Dimensão Diagnóstico':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('DIAG_PRINC', sa.VARCHAR(4)),
                    sa.Column('DIAG_SECUN', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC1', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC2', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC3', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC4', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC5', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC6', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC7', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC8', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC9', sa.VARCHAR(4)),
                    sa.Column('CID_MORTE', sa.VARCHAR(4)),
                    sa.Column('CID_ASSO', sa.VARCHAR(4))
            ).create(conn)
        else:
            raise ValueError('Essa tabela não é de dimensão conhecida.')
        
    def remove_tabela(self, conn, namedb):
        metadata = sa.MetaData()
        metadata.reflect(bind=conn)
        
        if namedb not in metadata.tables:
            print(f'Tabela {namedb} não existe.')
            return
        
        if self.dimensao == 'Dimensão Tempo':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('ANO_CMPT', sa.VARCHAR(4)),
                    sa.Column('MES_CMPT', sa.VARCHAR(2)),
                    sa.Column('DT_INTER', sa.VARCHAR(8)),
                    sa.Column('DT_SAIDA', sa.VARCHAR(8))
            ).drop(conn)
        elif self.dimensao == 'Dimensão Localização':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('CEP', sa.VARCHAR(8)),
                    sa.Column('MUNIC_RES', sa.VARCHAR(6)),
                    sa.Column('MUNIC_MOV', sa.VARCHAR(6))
            ).drop(conn)
        elif self.dimensao == 'Dimensão Hospital':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('CGC_HOSP', sa.VARCHAR(16)),
                    sa.Column('CNES', sa.VARCHAR(7))
            ).drop(conn)
        elif self.dimensao == 'Dimensão Paciente':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('NASC', sa.VARCHAR(8)),
                    sa.Column('SEXO', sa.VARCHAR(1)),
                    sa.Column('IDADE', sa.NUMERIC(2)),
                    sa.Column('COD_IDADE', sa.VARCHAR(1)),
                    sa.Column('NACIONAL', sa.VARCHAR(3)),
                    sa.Column('INSTRU', sa.VARCHAR(1)),
                    sa.Column('RACA_COR', sa.VARCHAR(2)),
                    sa.Column('ETNIA', sa.VARCHAR(4)),
                    sa.Column('CBOR', sa.VARCHAR(6)),
                    # sa.Column('CBOR', sa.VARCHAR(3)),
                    sa.Column('MORTE', sa.NUMERIC(1))
            ).drop(conn)
        elif self.dimensao == 'Dimensão UTI':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('UTI_MES_TO', sa.NUMERIC(3)),
                    sa.Column('MARCA_UTI', sa.VARCHAR(2)),
                    sa.Column('VAL_UTI', sa.NUMERIC(9,2))
            ).drop(conn)
        elif self.dimensao == 'Dimensão Procedimento':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('PROC_SOLIC', sa.VARCHAR(10)),
                    sa.Column('PROC_REA', sa.VARCHAR(10)),
                    sa.Column('VAL_SH', sa.NUMERIC(13,2)),
                    sa.Column('VAL_SP', sa.NUMERIC(13,2)),
                    sa.Column('N_AIH', sa.VARCHAR(13)),
                    sa.Column('VAL_TOT', sa.NUMERIC(14,2)),
                    sa.Column('INFEHOSP', sa.VARCHAR(1))
            ).drop(conn)
        elif self.dimensao == 'Dimensão Diagnóstico':
            sa.Table(namedb, sa.MetaData(),
                    sa.Column('DIAG_PRINC', sa.VARCHAR(4)),
                    sa.Column('DIAG_SECUN', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC1', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC2', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC3', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC4', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC5', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC6', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC7', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC8', sa.VARCHAR(4)),
                    sa.Column('DIAGSEC9', sa.VARCHAR(4)),
                    sa.Column('CID_MORTE', sa.VARCHAR(4)),
                    sa.Column('CID_ASSO', sa.VARCHAR(4))
            ).drop(conn)
        else:
            raise ValueError('Essa tabela não é de dimensão conhecida.')
        
    def limpando_tabela(self, conn, namedb):
        metadata = sa.MetaData()
        metadata.reflect(bind=conn)
        
        if namedb not in metadata.tables:
            print(f'Tabela {namedb} não existe.')
            return
        
        conn.execute(f"DELETE FROM {namedb}")

### Criando a conexão

In [ ]:
## CONECTANDO PELO WSL

# Configurações do banco de dados presentes no meu computador
usuario = 'postgres'
senha = '123,258D'
host = 'localhost'
porta = '5432'
banco_de_dados = 'my_test'

# URL de conexão
url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}"

# Criar o engine
engine = sa.create_engine(url)

# Teste a conexão
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")
finally:
    connection.close()


In [ ]:
# CONECTANDO PELO BANCO DO WINDOWS

# Configurações do banco de dados presentes no meu computador
usuario = 'postgres'
senha = 'dan313852*'
host = 'localhost'
porta = '5432'
banco_de_dados = 'fato_hospital'

# URL de conexão
url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}"

# Criar o engine
engine = sa.create_engine(url)

# Teste a conexão
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")
finally:
    connection.close()


### Exemplo para a dimensão de tempo

In [ ]:
table_tempo = tabelas('Dimensão Tempo')

df_tempo = df.filter(items = table_tempo.get_colunas())

df_tempo.head(10)

In [ ]:
table_tempo.remove_tabela(engine, 'tempo')

In [ ]:
# Criando a tabela no banco de dados
table_tempo.cria_tabela(engine, 'tempo')

In [ ]:
# Inserindo dados na tabela criada
table_tempo.insere_dados(df_tempo.head(10), engine, 'tempo')

### Fazendo para as demais dimensões

In [ ]:
table_tempo = tabelas('Dimensão Tempo')
table_localizacao = tabelas('Dimensão Localização')
table_hospital = tabelas('Dimensão Hospital')
table_paciente = tabelas('Dimensão Paciente')
table_uti = tabelas('Dimensão UTI')
table_procedimento = tabelas('Dimensão Procedimento')
table_diagnostico = tabelas('Dimensão Diagnóstico')

Removendo as tabelas para fazer testes.

In [ ]:
table_tempo.remove_tabela(engine, 'tempo')
table_localizacao.remove_tabela(engine, 'localizacao')
table_hospital.remove_tabela(engine, 'hospital')
table_paciente.remove_tabela(engine, 'paciente')
table_uti.remove_tabela(engine, 'uti')
table_procedimento.remove_tabela(engine, 'procedimento')
table_diagnostico.remove_tabela(engine, 'diagnostico')

In [ ]:
df_tempo = df.filter(items = table_tempo.get_colunas())
df_localizacao = df.filter(items = table_localizacao.get_colunas())
df_hospital = df.filter(items = table_hospital.get_colunas())
df_paciente = df.filter(items = table_paciente.get_colunas())
df_uti = df.filter(items = table_uti.get_colunas())
df_procedimento = df.filter(items = table_procedimento.get_colunas())
df_diagnostico = df.filter(items = table_diagnostico.get_colunas())

In [ ]:
table_tempo.cria_tabela(engine, 'tempo')
table_localizacao.cria_tabela(engine, 'localizacao')
table_hospital.cria_tabela(engine, 'hospital')
table_paciente.cria_tabela(engine, 'paciente')
table_uti.cria_tabela(engine, 'uti')
table_procedimento.cria_tabela(engine, 'procedimento')
table_diagnostico.cria_tabela(engine, 'diagnostico')



In [ ]:
df_paciente.head(10)

In [ ]:
table_localizacao.insere_dados(df_localizacao.head(10), engine, 'localizacao')
table_hospital.insere_dados(df_hospital.head(10), engine, 'hospital')
table_paciente.insere_dados(df_paciente.head(10), engine, 'paciente')
table_uti.insere_dados(df_uti.head(10), engine, 'uti')
table_procedimento.insere_dados(df_procedimento.head(10), engine, 'procedimento')
table_diagnostico.insere_dados(df_diagnostico.head(10), engine, 'diagnostico')